从7011中保存的generator参数中恢复模型，并生成图片

In [1]:
import tensorflow as tf
import multiprocessing
import os
import numpy as np
import glob
# scipy1.3.0  中没有 scipy.misc.toimage 方法，scipy1.0.0 中有，可以安装指定版本
from scipy.misc import toimage  # scipy.misc 中缺少toimage方法，用PIL中的Image.fromarray代替
from PIL import Image 
from tensorflow import keras
from tensorflow.keras import layers

from tensorflow.compat.v1 import ConfigProto
from tensorflow.compat.v1 import InteractiveSession

In [2]:
config = ConfigProto()   
config.gpu_options.allow_growth = True
session = InteractiveSession(config=config)
tf.random.set_seed(22)
np.random.seed(22)
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '2'
assert tf.__version__.startswith('2.')
assert np.__version__.startswith('1.16.2')

In [3]:
# 定义生成器G   向量[b, 100] => 图片[b, 81, 81, 3]
class Generator(keras.Model):
    def __init__(self):
        super(Generator, self).__init__()
        # 输入一个向量，经过生成器，变成一个图片，图片逐渐变大，通道数不断减少
        # z:[b, 100] => [b, 3*3*512]  全连接层        
        self.fc = layers.Dense(3*3*512)
        
        # [b, 3, 3, 512] => [b, 9, 9, 256]   反卷积层(输出通道数_卷积核数量；核的大小；步长；padding)
        # Conv2DTranspose 是考虑什么样的feature_map通过这个卷积层变成了现在的feature_map
        self.conv1 = layers.Conv2DTranspose(256, 3, 3, 'valid')
        self.bn1 = layers.BatchNormalization()
        # [b, 9, 9, 256] => [b, 29, 29, 128]  此处不是[b, 27, 27, 128]可能是因为受卷积核大小的影响吧
        self.conv2 = layers.Conv2DTranspose(128, 5, 3, 'valid')
        self.bn2 = layers.BatchNormalization()
        # [b, 29, 29, 128] => [b, 88, 88, 3]
        self.conv3 = layers.Conv2DTranspose(3, 4, 3, 'valid')
        
        
    def call(self, inputs, training=None):
        # [z, 100] => [z, 3*3*512]
        x = self.fc(inputs)
#         print(x.shape)
        
        # 获得图片feature_map
        x = tf.reshape(x, [-1, 3, 3, 512])        
        # 通过leaky_relu() 函数
        x = tf.nn.leaky_relu(x)
#         print(x.shape)
        
        # [b, 3, 3, 512] => [b, 9, 9, 256]
        x = tf.nn.leaky_relu(self.bn1(self.conv1(x), training=training))
#         print(x.shape)
        
        # [b, 9, 9, 256] => [b, 29, 29, 128]  因为卷积核比较大，是(5*5)所以得到是29*29
        x = tf.nn.leaky_relu(self.bn2(self.conv2(x), training=training))
#         print(x.shape)
        
        # [b, 29, 29, 128] => [b, 88, 88, 3]
        x = self.conv3(x)
        x = tf.tanh(x)
        
        return x    # [b, 88, 88, 3]
    

In [4]:
# 保存图片的函数save_result()
# save_result(fake_image.numpy(), 10, img_path, color_mode='P')    # [100, 88, 88, 3] 在 -1 到 1 之间
def save_result(val_out, val_block_size, image_path, color_mode):
    def preprocess(img):
        img = ((img + 1.0) * 127.5).astype(np.uint8)    # 变换到（0， 255），再转换数据类型为np.uint8
        # img = img.astype(np.uint8)
        return img

    preprocesed = preprocess(val_out)    # 获得经过preprocess()处理过的图片数据，在（0,255）之间     
    single_row = np.array([])     # 定义一行
    final_image = np.array([])    # 每次添加 single_row 一行的数据
    for b in range(val_out.shape[0]):    # val_out.shape[0] = 100,一张一张处理图片；b = 0, 1, ... 99
        # concat image into a row
        if single_row.size == 0:
            single_row = preprocesed[b, :, :, :] # preprocessed[0]表示第b张图片，并添加到single_row数组
        else:
            single_row = np.concatenate((single_row, preprocesed[b, :, :, :]), axis=1)

        # concat image row to final_image
        if (b+1) % val_block_size == 0:   # 每val_block_size（此处是10）张截取一行，当b=9时执行
            if final_image.size == 0:    
                final_image = single_row    # 把 single_row 中的数据拷贝至 final_image
            else:    # axis=0 表示 每次添加一行图片数据
                final_image = np.concatenate((final_image, single_row), axis=0)

            # reset single row
            single_row = np.array([])

    if final_image.shape[2] == 1:    # 防止 final_image 出现 [10, 10, 1]情况，squeeze 高维度的1
        final_image = np.squeeze(final_image, axis=2)
    
    toimage(final_image).save(image_path)


In [13]:
z_dim = 100
r_generator = Generator()
# 加载文件中的数据（保存有链接权重的数据）
r_generator.load_weights('/home/kukafee/workspace/save_model/GAN/G_weight_30000.ckpt')
# 打印信息
print('Loaded weights !')

Loaded weights !


从平均分布中sample数据（训练的时候也是从正态分布中sample的随机数据）

In [17]:
for i in range(10):
    z = tf.random.uniform([100, z_dim])    # [100, 100]
    fake_image = r_generator(z, training=False)    # [100, 88, 88, 3]
    # '/home/kukafee/workspace/picture/GAN/fake_epoch_%d.png'%epoch
    img_path = os.path.join('/home/kukafee/workspace/picture/R_GAN', 'fake_epoch_%d.png'%i)
    save_result(fake_image.numpy(), 10, img_path, color_mode='P')

/home/kukafee/environments/tf2_py3/lib/python3.6/site-packages/ipykernel_launcher.py:32: DeprecationWarning: `toimage` is deprecated!
`toimage` is deprecated in SciPy 1.0.0, and will be removed in 1.2.0.
Use Pillow's ``Image.fromarray`` directly instead.


从高斯分布中sample数据

In [19]:
for i in range(10):
    z = tf.random.normal([100, z_dim])    # [100, 100]
    fake_image = r_generator(z, training=False)    # [100, 88, 88, 3]
    # '/home/kukafee/workspace/picture/GAN/fake_epoch_%d.png'%epoch
    img_path = os.path.join('/home/kukafee/workspace/picture/R_GAN', 'normal_fake_epoch_%d.png'%i)
    save_result(fake_image.numpy(), 10, img_path, color_mode='P')

/home/kukafee/environments/tf2_py3/lib/python3.6/site-packages/ipykernel_launcher.py:32: DeprecationWarning: `toimage` is deprecated!
`toimage` is deprecated in SciPy 1.0.0, and will be removed in 1.2.0.
Use Pillow's ``Image.fromarray`` directly instead.
